# Retrieving prod models

In [ ]:
from mlflow.tracking.client import MlflowClient
import mlflow.pyfunc
import mlflow
import warnings
warnings.filterwarnings("ignore")

model_name = "BestModel"
model_stage = "Production"

mlflow.tracking.set_tracking_uri("http://127.0.0.1:5000")

client = MlflowClient()
model_version = client.get_latest_versions(model_name, stages=[model_stage])
model_version

In [ ]:
model_version[0].source

In [ ]:
model = mlflow.pyfunc.load_model(model_version[0].source)
model

In [ ]:
import pandas as pd
pd.plotting.register_matplotlib_converters()

input_data = pd.read_csv("germany.csv", parse_dates=[0], index_col=0)["2015-01"]
X_test = input_data[["windspeed", "temperature", "rad_horizontal", "rad_diffuse"]]
y_test = input_data[["solar_GW", "wind_GW"]]
x = X_test.index

In [ ]:
# This is the magic! We have a generic model from mlflow models that we can predict on. 
# We dont know or care about whether it was linear regression, RF, other, we just call predict
y_predict = model.predict(X_test)

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots(nrows=2, figsize=(9, 5), sharex=True)
axes[0].plot(x, y_test["solar_GW"], label="Actual")
axes[0].plot(x, y_predict[:, 0], label="Predicted")
axes[1].plot(x, y_test["wind_GW"], label="Actual")
axes[1].plot(x, y_predict[:, 1], label="Predicted")
axes[1].legend(), axes[0].set_ylabel("Solar GW"), axes[1].set_ylabel("Wind GW");